In [ ]:
!pip install casadi
import os
from IPython.display import clear_output

import numpy as np
import casadi
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib import patches

import os
os.makedirs('images',exist_ok=True)
if os.name == 'nt':
    plt.rcParams['font.family'] = 'MS Gothic'
elif os.name == 'posix':
    !pip install japanize-matplotlib
    import japanize_matplotlib

clear_output()

In [ ]:
x = casadi.SX.sym('x')
x

In [ ]:
z = casadi.SX.sym('Z',4,2)
print(z)


In [ ]:
print(casadi.sqrt(x**2 + 10))

In [ ]:
M = casadi.SX(casadi.diag([2,3,4,5]))
print(M)


In [ ]:
print(M[:2,:3])

In [ ]:
M[0,:] = 2
print(M)

In [ ]:
x = casadi.SX.sym('x')
y = casadi.SX.sym('y',2,2)
print(casadi.sin(y)-x)

In [ ]:
print(y*y)
print(y@y)

In [ ]:
x = casadi.SX.sym('x',2,2)
casadi.dot(x,y)

In [ ]:
x = casadi.SX.sym('x',1)
df = casadi.jacobian(x**2, x)
print(df)
print(casadi.simplify(df))

In [ ]:
A = casadi.SX.sym('A',3,2)
x = casadi.SX.sym('x',2)
print(A)
print(casadi.jacobian(A@x,x))

In [ ]:
[H,g] = casadi.hessian(casadi.dot(x,x),x)
print('H:', H)

In [ ]:
x = casadi.SX.sym('x')
y = casadi.SX.sym('y')
f1 = x
f2 = casadi.sin(y)*x
f = casadi.Function('f',[x,y],\
           [f1,f2],\
            ["x","y"],["f1","f2"])
print(f)

In [ ]:
print(f(x=1.1,y=2.1))

# 数理最適化

## NLP

In [ ]:
x = casadi.SX.sym('x')
y = casadi.SX.sym('y')
nlp = {'x':casadi.vertcat(x,y), \
        'f':x**3 + x**2 + 8*x + 4*y**2 + 3*x*y,\
        'g':x**2 + y**2 - 1}
S = casadi.nlpsol('S', 'ipopt', nlp)
print(S)

r = S(x0=[0, 1],\
      lbg=0, ubg=0)
x_opt = np.array(r['x'])
print('x_opt: ', x_opt)

X_, Y_ = np.arange(-2,2.01,0.01), np.arange(-2,2.01,0.01)
X, Y = np.meshgrid(X_, Y_)
Z = X**3 + X**2 + 8*X + 4*Y**2 + 3 * X * Y

levs = np.linspace(-20,50,20)

fig, ax = plt.subplots(figsize=(8,6))
ax.scatter(x_opt[0], x_opt[1], c="red")
cs = ax.contour(X,Y,Z,levels=levs)
ax.add_patch(plt.Circle( xy=(0, 0), radius=1, fill=False))
fig.colorbar(cs)

plt.savefig("images/chap3_NLP_2D.png")
plt.show()

## NLP example

In [ ]:
a = 10

R = casadi.SX.sym("R")
x_list = [casadi.SX.sym(f"x_{i}") for i in range(5)]
y_list = [casadi.SX.sym(f"y_{i}") for i in range(5)]
r_list = [R,R,R,2*R,2*R]

x_nlp = casadi.vertcat(R,*x_list,*y_list)
f_nlp = -R
g_nlp = []

g_nlp.append(R)

for i in range(5):
       g_nlp.append(x_list[i]-r_list[i])
       g_nlp.append(a-x_list[i]-r_list[i])
       g_nlp.append(y_list[i]-r_list[i])
       g_nlp.append(a-y_list[i]-r_list[i])

for j in range(5):
       for i in range(0,j):
              eq = (x_list[i]-x_list[j])**2 + (y_list[i]-y_list[j])**2 - (r_list[i]+r_list[j])**2
              g_nlp.append(eq)

g_nlp = casadi.vertcat(*g_nlp)

nlp = {"x":x_nlp,"f":f_nlp,"g":g_nlp}

S = casadi.nlpsol('S', 'ipopt', nlp)
print(S)

# x0=[1,1,3,8,7,3,1,2,8,3,7]→x0=[1,1,3,8,7,3,1,3,8,3,7]などとして、初期値依存性を確認する。
r = S(x0=[1,1,3,8,7,3,1,2,8,3,7],\
      lbg=[0]*31, ubg=[np.inf]*31)
x_opt = np.array(r['x']).ravel()
print('x_opt: ', x_opt)

R_opt = x_opt[0]
coord_opt = x_opt[1:].reshape(2,5).T

fig, ax = plt.subplots(figsize=(6,6))

for i in range(5):
       radius = R_opt if i <= 2 else 2*R_opt
       circle = patches.Circle(xy=coord_opt[i],radius=radius,fill=False)
       ax.add_patch(circle)
       ax.scatter(*coord_opt[i],marker="x")

rect = patches.Rectangle([0,0],10,10,fill=False)
ax.add_patch(rect)

ax.set_xlim(0,10)
ax.set_ylim(0,10)
ax.axis("equal")

plt.savefig("images/chap3_NLP_packing.png")
plt.show()

## QP

In [ ]:
x = casadi.SX.sym('x')
y = casadi.SX.sym('y')
qp = {'x':casadi.vertcat(x,y), \
        'f':x**2 + 2*y**2 - x - 2*y,\
        'g':casadi.vertcat(
            x+y-3,
            x-1,
            3-y,
            -x+3*y+1,
            5-x-y
        )}
S = casadi.qpsol('S', 'osqp', qp)
print(S)

r = S(x0=[0, 1],\
      lbg=[0,0,0,0,0], ubg=[0,np.inf,np.inf,np.inf,np.inf])
x_opt = np.array(r['x'])
print('x_opt: ', x_opt)

X_, Y_ = np.arange(0,4.01,0.01), np.arange(0,3.51,0.01)
X, Y = np.meshgrid(X_, Y_)
Z = X**2 + 2*Y**2 - X - 2*Y

levs = np.linspace(-20,30,50)

fig, ax = plt.subplots(figsize=(8,6))
ax.scatter(x_opt[0], x_opt[1], c="red")
cs = ax.contour(X,Y,Z,levels=levs)
fig.colorbar(cs)

points = [[1,0], [1,2], [1,3], [2,3], [4,1], [5/2,1/2]]
polygon = patches.Polygon(xy=points, closed=True,alpha=0.5)
ax.add_patch(polygon)

polyline = patches.Polygon([[1,2], [5/2,1/2]], closed=False, edgecolor='black', facecolor='none', linewidth=2)
ax.add_patch(polyline)

plt.savefig("images/chap3_QP_2D.png")
plt.show()

# ODE

In [ ]:
dt = 0.1 
t_eval = np.arange(0, 10+dt, dt)
X = [casadi.DM([1,1])]

A = casadi.DM([
    [0,1],
    [-10,-2]
])
x = casadi.SX.sym('x',2)

ode = A@x
dae = {"x":x, "ode":ode}

F = casadi.integrator('F','idas',dae,0,dt)

for t in t_eval[:-1]:
    res = F(x0=X[-1])
    X.append(res["xf"])

X = np.array(X).reshape(-1,2)

X_true_1 = casadi.exp(-t_eval)*casadi.cos(3*t_eval)+2/3*casadi.exp(-t_eval)*casadi.sin(3*t_eval)
X_true_1 = X_true_1.full().ravel()
X_true_2 = casadi.exp(-t_eval)*casadi.cos(3*t_eval)-11/3*casadi.exp(-t_eval)*casadi.sin(3*t_eval)
X_true_2 = X_true_2.full().ravel()

plt.plot(t_eval, X[:,0], label="数値解_x1", color="blue")
plt.plot(t_eval, X[:,1], label="数値解_x2", color="purple")
plt.plot(t_eval, X_true_1, label="解析解_x1", color="red",linestyle="--")
plt.plot(t_eval, X_true_2, label="解析解_x2", color="green",linestyle="--")
plt.legend()
plt.savefig("images/chap3_integ.png")
plt.show()    

In [ ]:
# x = casadi.SX.sym('x')
# dt = 0.1 
# times = np.arange(0, 2+dt, dt)
# X_t = [1] # 初期値


# options = {'t0':0, 'tf':dt} # 積分範囲（時刻）
# ode = {'x': x, 'ode': -2*x} # 常微分方程式

# F = casadi.integrator('F', 'idas', ode, options)

# for t in times[:-1]:
#     res = F(x0=X_t[-1])
#     X_t += res['xf'].toarray().tolist()[0]

# Y_t = np.exp(-2*times)  #真の解     

# plt.plot(times, X_t, label="数値解", color="blue")
# plt.plot(times, Y_t, label="解析解", color="red")
# plt.legend()
# plt.savefig("images/chap3_integ.png")
# plt.show()    